## Transfer learning & Fine tuning with Tensorflow on Dogs & Cats

---

In [1]:
# Downloading the Dogs vs Cats dataset 

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-11-18 13:54:10--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:4000:804::2010, 2607:f8b0:4000:803::2010, 2607:f8b0:4000:80f::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:4000:804::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  37.1MB/s    in 1.8s    

2020-11-18 13:54:12 (37.1 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
# Import project dependencies

import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [4]:
# Unzipping the Dogs vs Cats dataset

dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [5]:
# Seting up dataset paths

dataset_path_new = "./cats_and_dogs_filtered/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [7]:
# Building the model
# Loading the pre-trained model (MobileNetV2)

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
________________

In [8]:
# Freezing the base model

base_model.trainable = False

In [9]:
# Defining the custom head for network

base_model.output

<tf.Tensor 'out_relu/Relu6_1:0' shape=(None, 4, 4, 1280) dtype=float32>

In [10]:
# Defining the global average layer

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [11]:
# Defining the perdiction layer

prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [12]:
# Defining the model

model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

In [13]:
# Compiling the model

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [14]:
# Resizing images, Big pre-trained architecture support only certain input sizes.
# For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [15]:
# Creating train generator

train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [16]:
# Creating valid generator

valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [17]:
# Training the model

model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
16/16 [==============================] - 29s 2s/step - loss: 0.7355 - accuracy: 0.5530 - val_loss: 0.6707 - val_accuracy: 0.6160
Epoch 2/5
16/16 [==============================] - 30s 2s/step - loss: 0.6224 - accuracy: 0.6670 - val_loss: 0.5821 - val_accuracy: 0.7230
Epoch 3/5
16/16 [==============================] - 32s 2s/step - loss: 0.5419 - accuracy: 0.7475 - val_loss: 0.5099 - val_accuracy: 0.7760
Epoch 4/5
16/16 [==============================] - 31s 2s/step - loss: 0.4763 - accuracy: 0.8075 - val_loss: 0.4501 - val_accuracy: 0.8190
Epoch 5/5
16/16 [==============================] - 33s 2s/step - loss: 0.4215 - accuracy: 0.8415 - val_loss: 0.4006 - val_accuracy: 0.8560


In [18]:
# Transfer learning model evaluation

valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Instructions for updating:
Please use Model.evaluate, which supports generators.
Accuracy after transfer learning: 0.8560000061988831


In [20]:
# Fine tuning

# Un-freeze a few top layers from the model

base_model.trainable = True
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [21]:
# Fine tune at 100

fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [22]:
# Compiling the model for fine-tuning

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
# Fine tuning

model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 44s 3s/step - loss: 0.2061 - accuracy: 0.9180 - val_loss: 0.1067 - val_accuracy: 0.9570
Epoch 2/5
16/16 [==============================] - 43s 3s/step - loss: 0.0288 - accuracy: 0.9980 - val_loss: 0.1575 - val_accuracy: 0.9510
Epoch 3/5
16/16 [==============================] - 43s 3s/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.1602 - val_accuracy: 0.9550
Epoch 4/5
16/16 [==============================] - 43s 3s/step - loss: 0.0058 - accuracy: 1.0000 - val_loss: 0.1675 - val_accuracy: 0.9570
Epoch 5/5
16/16 [==============================] - 43s 3s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.2133 - val_accuracy: 0.9530


In [24]:
# Evaluating the fine tuned model

valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.953000009059906
